# Wine Classification

In [ ]:
import numpy as np
import pandas as pd
import pickle
import tensorflow as tf
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import seaborn as sns
plt.style.use('fivethirtyeight')
print(tf.__version__)

In [ ]:
# Mount your Google Drive
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

In [ ]:
# Change to top directory of your Drive
import os
os.chdir('/content/gdrive/Shared drives/AI4ALL SFU NLP GROUP 3/WINE')

In [ ]:
# Load data
df = pd.read_csv('winemag-data-130k-v2.csv')

In [ ]:
df.head(10)

In [ ]:
# Get only the columns we want
wine_df = df[['description', 'variety']].copy()

In [ ]:
wine_df

In [ ]:
wine_df['variety'].value_counts()[:5]

In [ ]:
topVarieties = wine_df['variety'].value_counts()[:5].index.tolist()

In [ ]:
wine_df_short = wine_df[wine_df.variety.isin(topVarieties)].copy()

In [ ]:
wine_df_short

In [ ]:
wine_df_short['variety_num'] = wine_df_short['variety'].astype('category').cat.codes

In [ ]:
wine_df_short

In [ ]:
# Split our data into training and test sets (80/20)
train_df, test_df = train_test_split(wine_df_short, test_size=0.2)

In [ ]:
import string
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stops = stopwords.words('english')
for i in topVarieties:
  stops.extend(i.lower().split(' '))
print(stops[-15:])


In [ ]:
def cleanText(text):
    # Remove new lines from the text
    text = text.replace("\n", " ")
    text = text.lower()
    text = text.split(' ')
    text = [w for w in text if not w in stops] 
    text = ' '.join(text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = ''.join([i for i in text if not i.isdigit()])
    return text

In [ ]:
train_df['description'] = train_df['description'].apply(lambda x : cleanText(x))
test_df['description'] = test_df['description'].apply(lambda x : cleanText(x))

In [ ]:
train_df

In [ ]:
X_train = train_df['description']
y_train = train_df['variety_num']
X_test = test_df['description']
y_test = test_df['variety_num']

In [ ]:
vocab_size = 20000
max_seq_length = 400

In [ ]:
# TfidfVectorizer converts text to a matrix of TF-IDF features
from sklearn.feature_extraction.text import TfidfVectorizer

vect = TfidfVectorizer()

In [ ]:
''' 
TO DO: Define our X and y from the training set
(Re)load the (cleaned) training and test datasets as X_train, y_train, X_test, y_test
'''
# TO DO: Load our pre-processed data into a dataframe
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [ ]:
X_train_vect = vect.fit_transform(X_train)
X_test_vect = vect.transform(X_test)

In [ ]:
# Import classifiers from Scikit-learn
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier

In [ ]:
'''
TO DO: Define a list of models that we will be comparing. Choose from the ones
that were imported from above.
'''
NB = MultinomialNB()
SVC = LinearSVC()
RFC = RandomForestClassifier(max_depth=10)
models = [NB, SVC, RFC]

In [ ]:
# Create a dictionary for storing the results of each run for each model
results = {}

# For each model, create a dictionary within results that stores the accuracy,
# f1 score and confusion matrix values
for model in models:
    results[model.__class__.__name__] = {'accuracy': [], 
                                       'f1_score': [], 
                                       'confusion_matrix': []}

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

In [ ]:
for model in models:
    '''
    TO DO: For each model in our models list, we want to:

    1. Fit the model on X_train_vect and y_train

    2. Generate y_pred by calling model.predict
    '''
    model.fit(X_train_vect, y_train)
    y_pred = model.predict(X_test_vect)
    '''
    TO DO: Compute the following so we can add it to our results:
    acc - accuracy score
    f1  - f1 score
    cm - confusion matrix
    '''
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    cm = confusion_matrix(y_test, y_pred)
    print("done")
    results[model.__class__.__name__]['accuracy'] = acc
    results[model.__class__.__name__]['f1_score'] = f1
    results[model.__class__.__name__]['confusion_matrix'] = cm

In [ ]:
'''
TO DO: Display the results by printing out the accuracy, F1 score and confusion
matrix for each of the models.
'''
for model in results:
  print(results[model])